In [1]:
from PIL import Image
import requests
import numpy as np
from PIL import Image

import pandas as pd 
import geopandas as gpd
# import osmnx as ox
# import networkx as nx

import pyproj
from shapely.ops import transform

import shapely
from shapely import Point

# import boto3
import os
import django

pd.set_option('display.precision', 2)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "water.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from water.models import ReservoirState, Reservoir, ReservoirStateSerializer

rs = ReservoirState.objects.all()[0]
ReservoirStateSerializer(rs).data

{'uuid': '79ccacdc-a44b-482f-9490-69df36d4d787', 'date': '2024-01-01', 'volume': 2.18, 'reservoir': {'uuid': 'd6cc293e-9179-4d24-9d55-e987122032dc', 'name': 'almodovar', 'capacity': 5.7}}

AssertionError: The field 'reservoir_reservoirstate' was declared on serializer ReservoirStateSerializer, but has not been included in the 'fields' option.

In [4]:
rs.reservoir.name

'almodovar'

In [ ]:
# Load the data from ../data/datasets/all_parsed.csv

filename = '../../../../data/datasets/all_parsed.csv'
df_raw = pd.read_csv(filename)

df = df_raw.query('"2024-01-01" <= ds <= "2024-01-05"').copy()
reservoirs = df.reservoir.unique()

In [ ]:
capacities = df.groupby('reservoir')['capacity_hm3'].agg(['first', 'nunique']).reset_index()

assert capacities['nunique'].max() == 1

capacities

In [ ]:
ReservoirState.objects.all().delete()
Reservoir.objects.all().delete()

for _, row in capacities.iterrows():
    reservoir = Reservoir.objects.create(name=row['reservoir'], capacity=row['first'])
    reservoir.save()
    
for _, row in df.iterrows():
    reservoir = Reservoir.objects.get(name=row['reservoir'])
    state = ReservoirState.objects.create(
        reservoir=reservoir,
        date=row['ds'],
        volume=row['stored_hm3'],
    )
    state.save()

In [ ]:
ReservoirState.objects.all()[0:5]